In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install plotly
%pip install numpy
%pip install matplotlib
%pip install nbformat
%pip install opencv-python
%pip install seaborn
%pip install scikit-image
%pip install keras
%pip install tensorflow
%pip install wandb



In [1]:


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import plotly.express as px
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sn
import keras.backend as K
import wandb
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, vgg16
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop

import wandb
wandb.login()
%env "WANDB_NOTEBOOK_NAME" "model_unbalanced.ipynb"
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

import tensorflow as tf

from sklearn.metrics import f1_score


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jonas14 (dspro2-group9). Use `wandb login --relogin` to force relogin


env: "WANDB_NOTEBOOK_NAME"="model_unbalanced.ipynb"


In [2]:
emotion_labels = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral",
}

print(emotion_labels)

{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}


In [3]:
fer2013_df = pd.read_csv("../../../data/raw/fer2013.csv", index_col=False)
fer2013_df.drop(columns="Usage", inplace=True)

fer2013_df.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [4]:
for index, row in fer2013_df.iterrows():
    fer2013_df.loc[index, "emotion"] = emotion_labels.get(row["emotion"])

C:\Users\Jonas\AppData\Local\Temp\ipykernel_32252\3253472185.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Angry' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fer2013_df.loc[index, "emotion"] = emotion_labels.get(row["emotion"])


In [5]:
fer2013_df["emotion"] = fer2013_df["emotion"].astype("category")

In [6]:
y = pd.get_dummies(fer2013_df['emotion'])
y.drop(columns="Neutral",inplace=True)
y = y.values

In [8]:
X = []
for picture in fer2013_df["pixels"].values:
    image_string = picture.split(" ")
    image_data = np.asarray(image_string, dtype=np.uint8).reshape(48, 48)
    image_rgb = np.stack([image_data] * 3, axis=-1)  # Create 3-channel image
    image_rgb_normalized = image_rgb / 255.0  # Normalize pixel values
    X.append(image_rgb_normalized)


In [9]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=validation_ratio / (train_ratio + test_ratio)
)

In [10]:
X_train = np.asarray(X_train)
X_test= np.asarray(X_test)
X_valid= np.asarray(X_valid)
y_train = np.asarray(y_train)
y_test= np.asarray(y_test)
y_valid= np.asarray(y_valid)


X_test = vgg16.preprocess_input(X_test)
X_train = vgg16.preprocess_input(X_train)
X_valid = vgg16.preprocess_input(X_valid)

In [11]:


IMG_SIZE = (48, 48)


base_model = VGG16(
        weights=None,
        include_top=False, 
        input_shape=IMG_SIZE + (3,)
    )

base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
NUM_CLASSES = 6

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [13]:
def deep_model(model, X_train, Y_train, epochs, batch_size, validation_data):
   
    model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=['accuracy'])
    
    history = model.fit(X_train
                       , Y_train
                       , validation_data = validation_data
                       , epochs=epochs
                       , batch_size=batch_size
                       , verbose=1
                       ,callbacks=[WandbMetricsLogger(log_freq=5)])
    return history

In [14]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    name="testrun unbalanced fer dataset",


    # track hyperparameters and run metadata with wandb.config
    config={
        "optimizer": "RMSprop(learning_rate=1e-4)",
        "loss": "binary_crossentropy",
        "metric": ["accuracy"],
        "epoch": 50
    }
)

In [15]:
epochs = 50
batch_size = 32

history = deep_model(model, X_train, y_train, epochs, batch_size, (X_valid, y_valid))

wandb.finish()

Epoch 1/50


wandb: ERROR Unable to log learning rate.


223/898 ━━━━━━━━━━━━━━━━━━━━ 9:31 846ms/step - accuracy: 0.2259 - loss: 0.4447